<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [992]:
import pandas as pd
import psycopg2

In [994]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [995]:
# текст запроса
query = f'''select count (id) 
            from vacancies
         '''

In [996]:
res = pd.read_sql_query(query, connection)
res

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [997]:
# текст запроса
query = f'''select count (id)
            from employers
         '''

In [998]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [999]:
# текст запроса
query = f'''select count (id)
            from areas
         '''

In [1000]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [1001]:
# текст запроса
query = f'''select count (id)
            from industries
         '''

In [1002]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,294


***

# Выводы по предварительному анализу данных
В базе находится около 50 тыс. активных вакансий. Количество работодателей - 23,5 тыс. Количество зарегистрированных в базе сфер деятельности - около 300. Количество зарегистрированных городов - около 1 400. Однако, вероятно, не все зарегистрованные в базе работодатели имеют в настоящее время активные вакансии. Также пока не понятно, какая часть из зарегистрированных городов и сфер деятельности реально представлены в активных вакансиях.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [1003]:
# текст запроса
query = f'''select
                a.name,
                count (*) vacancies_total
            from vacancies v
            left join areas a on v.area_id = a.id
            group by 1
            order by 2 desc
            limit 5

         '''

In [1004]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,name,vacancies_total
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [1005]:
# текст запроса
query = f'''select
                count (*)
            from vacancies
            where salary_from is not null or salary_to is not null
         '''

In [1006]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [1007]:
# текст запроса
query = f'''select
                round(avg(salary_from),0),
                round(avg(salary_to),0)
            from vacancies
         '''

In [1008]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [1009]:
# текст запроса
query = f'''select
                schedule,
                employment,
                count(*)
            from vacancies
            group by 1,2
            order by 3 desc
         '''

In [1010]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [1011]:
# текст запроса
query = f'''select
                experience,
                count(*)
            from vacancies
            group by 1
            order by 2
         '''

In [1012]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

# выводы по детальному анализу вакансий
На топ-5 регионов (часть из них вне России) приходится около 14 тыс. вакансий (почти 30% от общего количества). К данным регионам относятся Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. 
Почти в половине вакансий (24 тыс.) отсутствует информация о заработной плате. В вакансиях, где зарплата указана, средняя зарплатная "вилка" по рынку труда в целом выглядит следующим образом: нижняя граница - 71 тыс. рублей, верхняя - 110,5 тыс. рублей.
Около 70% вакансий предполагают очный формат работы (полный день) с полной занятостью. Но удаленная работа занимает заметную долю вакансий (около 20%). Причем при частичной занятости удаленная работа более популярна чем, работа в очном формате. 
Распределение вакансий по требованиям работодателя к опыту работы выглядит следующим образом. Не требуется опыт работы примерно в 15% вакансий, минимальный опыт работы (от 1 до 3 лет) - требуется примерно у 52% вакансий, примерно 30% вакансий требуют средний опыт работы (от 3 до 6 лет), остальные вакансии (около 3%) требуют большой опыт работы (от 6 лет).

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [1013]:
# текст запроса
query = f'''(select
                e.name,
                count (*) vacancies_total
            from vacancies v
            left join employers e on v.employer_id = e.id
            group by 1
            order by 2 desc
            limit 1)
            
            union all
            (select
                e.name,
                count (*) vacancies_total
            from vacancies v
            left join employers e on v.employer_id = e.id
            group by 1
            order by 2 desc
            offset 4 limit 1)
         '''

In [1014]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,name,vacancies_total
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [1015]:
# текст запроса
query = f'''select
                q1.name,
                q1.employers_total,
                q2.vacancies_total
            from
            (select
                a.name,
                count (*) employers_total
            from employers e
            left join areas a on e.area = a.id
            group by 1
            order by 2 desc) q1
            left join
                (select
                    a.name,
                    count (*) vacancies_total
                from vacancies v
                left join areas a on v.area_id = a.id
                group by 1
                order by 2 desc) q2 on q1.name = q2.name
            order by 3 nulls first, 2 desc
            limit 1
             '''

In [1016]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,name,employers_total,vacancies_total
0,Россия,410,None


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [1017]:
query = f'''
            select
                q1.employer,
                count(*)
            from 
            (select
                e.name employer,
                a.name area,
                count(*) vacancies_count
            from vacancies v
            left join employers e on v.employer_id = e.id
            left join areas a on v.area_id = a.id
            group by 1,2
            order by 3 desc, 1,2) q1
            group by 1
            order by 2 desc
         '''

In [1018]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,employer,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,Интеркросс,1
14762,Русская дымка,1
14763,Автономная некоммерческая организация Московск...,1
14764,Транспортный холдинг города Алматы,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [1019]:
# текст запроса
query = f'''
        select 
            count(*)
        from
        (select
            e.id
        from employers e
        left join employers_industries ei on e.id = ei.employer_id
        where ei.employer_id is null) q1
        '''

In [1020]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [1021]:
# текст запроса
query = f'''
        select 
            q1.employer,
            count(*) industries_count
        from
        (select
            e.name employer
        from employers e
        left join employers_industries ei on e.id = ei.employer_id
        where ei.employer_id is not null) q1
        group by 1
        having count(*) = 4
        order by 1 offset 2 limit 1
        '''

In [1022]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,employer,industries_count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [1023]:
# текст запроса
query = f'''
        select
           count(*)
        from
        (select
            e.name employer,
            i.name
        from employers e
        left join employers_industries ei on e.id = ei.employer_id
        left join industries i on ei.industry_id=i.id
        where i.name = 'Разработка программного обеспечения') q1
        '''

In [1024]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [1025]:
# код для получения списка городов-милионников
import urllib.parse as req
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
cities = pd.read_html(url)[1]
cities = tuple(cities['Город'])

In [1026]:
# текст запроса
query = f'''
        select
            q3.area,
            q3.count 
        from
            (
                (select
                    'total' area, 
                    count(*) count
                from
                    (select
                        a.name area
                    from vacancies v
                    left join employers e on v.employer_id = e.id
                    left join areas a on v.area_id=a.id
                    where e.name = 'Яндекс' and a.name in {cities}
                    ) q1
                )
                union all
                (select
                    distinct q1.area,
                    count(*)
                from
                    (select
                        a.name area
                    from vacancies v
                    left join employers e on v.employer_id = e.id
                    left join areas a on v.area_id=a.id
                    where e.name = 'Яндекс' and a.name in {cities}
                    ) q1
                group by 1
                order by 2
                )
            ) q3
        order by 2, 1
    '''


In [1027]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,area,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Казань,25
5,Пермь,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

# выводы по анализу работодателей
Около 3.5 тыс. компаний-работодателей занимаются разработкой программного обеспечения. Компания "Яндекс" - самая представленная в регионах России компания.
В частности, компания "Яндекс" имеет 485 вакансий во всех городах-миллиониках России.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [1028]:
# текст запроса
query = f'''select 
                count(*)
            from vacancies
            where lower(name) LIKE '%data%' or lower(name) LIKE '%данн%'
         '''


In [1029]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [1030]:
# текст запроса
query = f'''SELECT
                COUNT(*)
           FROM 
                vacancies
           WHERE 
                (LOWER(name) LIKE ('%data scientist%') OR LOWER(name) LIKE ('%data science%') OR LOWER(name) LIKE ('%исследователь данных%') OR LOWER(name) LIKE ('%machine learning%') 
                OR LOWER(name) LIKE ('%машинн%обучен%') OR (LOWER(name) LIKE ('%ml%') AND LOWER(name) NOT LIKE ('%html%')))
                AND (LOWER(name) LIKE ('%junior%') OR LOWER(experience) = 'нет опыта' OR lower(employment) = 'стажировка')
         '''


In [1031]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [1032]:
# текст запроса
query = f'''SELECT 
                COUNT(*)
            FROM
                vacancies
            WHERE
                (LOWER(name) LIKE '%data scientist%'  OR lower(name) LIKE '%data science%' OR LOWER(name) LIKE '%исследователь данных%'  
                OR (name LIKE '%ML%' and name NOT LIKE '%HTML%')  OR LOWER(name) LIKE '%machine learning%'  or LOWER(name) LIKE '%машинн%обучен%')
                AND
                (LOWER(key_skills) LIKE '%sql%' OR LOWER(key_skills) LIKE '%postgres%')
         '''


In [1033]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [1034]:
# текст запроса
query = f'''SELECT 
                COUNT(*)
            FROM
                vacancies
            WHERE
                (LOWER(name) LIKE '%data scientist%'  OR lower(name) LIKE '%data science%' OR LOWER(name) LIKE '%исследователь данных%'  
                OR (name LIKE '%ML%' and name NOT LIKE '%HTML%')  OR LOWER(name) LIKE '%machine learning%'  or LOWER(name) LIKE '%машинн%обучен%')
                AND LOWER(key_skills) LIKE '%python%'
         '''

In [1035]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [1036]:
# текст запроса
query = f'''SELECT 
                ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1), 2)
            FROM vacancies
            WHERE
                (LOWER(name) LIKE '%data scientist%'  OR lower(name) LIKE '%data science%' OR LOWER(name) LIKE '%исследователь данных%'  
                OR (name LIKE '%ML%' and name NOT LIKE '%HTML%')  OR LOWER(name) LIKE '%machine learning%'  or LOWER(name) LIKE '%машинн%обучен%')
        '''

In [1037]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [1038]:
# текст запроса
query = f'''SELECT
                experience,
                CAST(
                    ROUND(
                            AVG(
                                CASE
                                    WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN 0.5 * (salary_to + salary_from)
                                    ELSE COALESCE(salary_to, salary_from)
                                END
                            )
                        )
                AS INT) AS "average salary"
            FROM 
                vacancies
            WHERE
                (LOWER(name) LIKE '%data scientist%'  OR lower(name) LIKE '%data science%' OR LOWER(name) LIKE '%исследователь данных%'  
                OR (name LIKE '%ML%' and name NOT LIKE '%HTML%')  OR LOWER(name) LIKE '%machine learning%'  or LOWER(name) LIKE '%машинн%обучен%')
                AND NOT (salary_from IS NULL AND salary_to IS NULL)
            GROUP BY experience
            ORDER BY "average salary"
        '''

In [1039]:
# результат запроса
res = pd.read_sql_query(query, connection)
res

,experience,average salary
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


***

# выводы по предметному анализу
Среди 50 тыс. вакансий так или иначе связаны с данными около 1,8 тыс. (примерно 4%). Из этого количества около 500 вакансий связаны со сферой Data Science, в том числе около 50 вакансий - для начинающих дата-сайентистов. 
В вакансиях, связанных со сферой Data Science, от соискателей требуется в среднем около 6,5 ключевых навыка. Среди часто встречающихся ключевых навыков, требуемых работодателями в сфере Data Science, можно выделить владение Python (примерно в 70% вакансий) и SQL (примерно в 40% ваканcий).
Зарплата специалистов в сфере Data Science в среднем существенно выше, чем по рынку труда в целом. Вот как различается зарплата на рынке труда в целом и в сфере Data Science в частности: 
* соискатели без опыта работы: по рынку труда в целом - 40,5 тыс. рублей, а в сфере Data Science - 74,6 тыс. рублей;
* соискатели с опытом работы от 1 до 3 лет: на рынке труда в целом - 76,5 тыс. рублей, a в сфере Data Science - 139,7 тыс. рублей; 
* соискатели c опытом работы от 3 до 6 лет: на рынке труда в целом - 133,2 тыс. рублей, а в сфере Data Science - 243,1 тыс. рублей.
* для соискателей с опытом работы более 6 лет в сфере Data Science зарплаты не указаны, но можно предположить, что и они существенно выше рынка труда в целом.


# Общий вывод по проекту

In [1044]:
query = f'''SELECT
                COUNT(*)
            FROM
                vacancies v
                left join areas a on v.area_id = a.id
            where
                a.name in {cities}
        '''

In [1045]:
res = pd.read_sql_query(query, connection)
res

,count
0,23856


In [1050]:
query = f'''SELECT 
                COUNT(*)
            FROM
                vacancies
            WHERE
                (LOWER(name) LIKE '%data scientist%'  OR lower(name) LIKE '%data science%' OR LOWER(name) LIKE '%исследователь данных%'  
                OR (name LIKE '%ML%' and name NOT LIKE '%HTML%')  OR LOWER(name) LIKE '%machine learning%'  or LOWER(name) LIKE '%машинн%обучен%')
                AND LOWER(key_skills) LIKE '%python%'
                AND experience = 'Более 6 лет'
         '''

In [1051]:
res = pd.read_sql_query(query, connection)
res

,count
0,20


In [1052]:
# текст запроса
query = f'''SELECT
                experience,
                CAST(
                    ROUND(
                            AVG(
                                CASE
                                    WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL THEN 0.5 * (salary_to + salary_from)
                                    ELSE COALESCE(salary_to, salary_from)
                                END
                            )
                        )
                AS INT) AS "average salary"
            FROM 
                vacancies
            WHERE
                NOT (salary_from IS NULL AND salary_to IS NULL)
            GROUP BY experience
            ORDER BY "average salary"
        '''

In [1053]:
res = pd.read_sql_query(query, connection)
res

,experience,average salary
0,Нет опыта,40472
1,От 1 года до 3 лет,76541
2,От 3 до 6 лет,133211
3,Более 6 лет,166256


# Общие выводы
* Проанализированы около 50 тыс. вакансий из базы данных компании HeadHunter. В HeadHunter работодателей - 23,5 тыс. из около 300 сфер деятельности. Количество зарегистрированных городов - около 1 400.
* В Москве, Санкт-Петербурге, Минске, Новосибирске и Алмате находятся около 14 тыс. вакансий (почти 30% от общего количества). Города-миллионники России и крупные зарубежные города занимают более половины всех вакансий на рынке труда.
* Почти в половине вакансий (24 тыс.) отсутствует информация о заработной плате. Интересно проанализировать данные вакансии по сферам деятельности, крупным работодателям, ключевым навыкам, опыту работы соискателей, чтобы сделать обоснованные предположения о наиболее вероятных причинах неуказания зарплаты работодателями. 
* В вакансиях, где зарплата указана, средняя зарплатная "вилка" по рынку труда в целом выглядит следующим образом: нижняя граница - 71 тыс. рублей, верхняя - 110,5 тыс. рублей.
Помимо средней зарплатной вилки, можно вычислить медианную зарплатную вилку, чтобы оценить насколько высоко и/или низкооплачиваемые позиции влияют на средний уровень зарплат. 
* По-прежнему полная занятость с очным присутствием на рабочем месте доминирует на рынке труда. Около 70% вакансий предполагают именно такое сочетание. Но удаленная работа занимает уже заметную долю вакансий (около 20%). Причем при частичной занятости удаленная работа более популярна чем, работа в очном формате. Остальные графики работы и виды занятости представлены на рынке труда гораздо слабее. 
* Распределение вакансий по требованиям работодателя к опыту работы выглядит следующим образом. Не требуется опыт работы примерно в 15% вакансий, минимальный опыт работы (от 1 до 3 лет) - требуется примерно у 52% вакансий, примерно 30% вакансий требуют средний опыт работы (от 3 до 6 лет), остальные вакансии (около 3%) требуют большой опыт работы (от 6 лет). Скорей всего, такое распределение связано с тем, что вакансии, требующие более высокий опыт работы, - это более высокопоставленные позиции, которые по определению более малочисленны. Кроме того, часть работодателей предпочитают закрывать такие вакансии из внутренних кандидатов без выхода на рынок труда.

* Среди 50 тыс. вакансий так или иначе связаны с данными около 1,8 тыс. (примерно 4%). Из этого количества около 500 вакансий (т.е. около 1% от всех вакансий) связаны со сферой Data Science, из них около 50 - вакансии для начинающих дата-сайентистов. 

* В вакансиях, связанных со сферой Data Science, от соискателей требуется большое количество навыков. Среди часто встречающихся ключевых навыков, требуемых работодателями в сфере Data Science, можно выделить владение Python (примерно в 70% вакансий) и SQL (примерно в 40% ваканcий). Интересно дополнительно проанализировать востребованность остальных навыков, а также их распределение по должностным уровням (junior, middle, senior).
Зарплата специалистов в сфере Data Science в среднем существенно выше, чем по рынку труда в целом (практически в 2 раза). 
Вот как различается зарплата на рынке труда в целом и в сфере Data Science в частности: 
* соискатели без опыта работы: по рынку труда в целом - 40,5 тыс. рублей, а в сфере Data Science - 74,6 тыс. рублей;
* соискатели с опытом работы от 1 до 3 лет: на рынке труда в целом - 76,5 тыс. рублей, a в сфере Data Science - 139,7 тыс. рублей; 
* соискатели c опытом работы от 3 до 6 лет: на рынке труда в целом - 133,2 тыс. рублей, а в сфере Data Science - 243,1 тыс. рублей.
* для соискателей с опытом работы более 6 лет в сфере Data Science зарплаты не указаны ни в одной вакансии. Таких вакансий всего 20, т.е. примерно 4% от общего количества вакансий в сфере Data Science, что в целом соответствует общим пропорциям на рынке труда. И ни в одной из этих вакансий заработная плата не указана. Поскольку средняя заработная плата для соискателей с данным опытом работы в целом по рынку труда составляет 166,3 тыс. рублей, среднюю зарплату дата-сайентиста с опытом работы от 6 лет можно первоначально оценить в интервале 300-350 тыс. рублей (конечно, данная гипотеза требует более обоснованного подтверждения).